In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
%matplotlib inline

In [2]:
dfLE = pd.read_csv("https://raw.githubusercontent.com/fenago/introml/main/Life%20Expectancy%20Data.csv")

In [3]:
dfLE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

In [4]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import certifi

In [5]:
from sklearn.datasets import fetch_openml

In [6]:
# Download the datasets from OPENML
df = fetch_openml(data_id=42803, as_frame=True)

In [7]:
data = df

In [8]:
# Extract feature matrix X and show a few samples
df_X = data["frame"]
df_X.shape

(363243, 67)

In [9]:
df_X.shape

(363243, 67)

In [10]:
pd.set_option('max_columns',None)

In [11]:
df_X.nunique().to_frame().T

,Accident_Index,Vehicle_Reference_df_res,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,1st_Point_of_Impact,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_Home_Area_Type,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Vehicle_Reference_df,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,140056,37,20,6,18,10,9,6,12,9,12,5,2,6,3,95,11,1140,11,72,3,78609,82205,117588,116292,49,3,15,21,365,7,1439,356,204,6,3938,6,8,9,4,6,3707,3,6,5,9,5,8,6,2,3,25979,28,38,3,2,101,11,3,11,10,3,5,3,21,3,10


In [12]:
# Count of how many times each data type is present in the dataset
pd.value_counts(df_X.dtypes)

float64    61
object      6
dtype: int64

In [13]:
df_X.dtypes.to_frame().T

,Accident_Index,Vehicle_Reference_df_res,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,1st_Point_of_Impact,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_Home_Area_Type,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Vehicle_Reference_df,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [14]:
df_X.head()

,Accident_Index,Vehicle_Reference_df_res,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,1st_Point_of_Impact,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_Home_Area_Type,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Vehicle_Reference_df,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,201501BS70001,1.0,19.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,2143.0,2.0,4.0,NaN,525130.0,180050.0,-0.198465,51.505538,1.0,3.0,1.0,1.0,12/01/2015,2.0,18:45,12.0,E09000020,5.0,0.0,6.0,30.0,3.0,4.0,6.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,1.0,1.0,E01002825,1.0,1.0,3.0,2.0,41.0,7.0,3.0,5.0,1.0,0.0,0.0,2.0,0.0,NaN,NaN
1,201501BS70002,1.0,9.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,NaN,NaN,1600.0,1.0,3.0,NaN,526530.0,178560.0,-0.178838,51.491836,1.0,3.0,1.0,1.0,12/01/2015,2.0,07:50,12.0,E09000020,6.0,0.0,6.0,30.0,3.0,4.0,3.0,3218.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,E01002820,1.0,1.0,3.0,1.0,24.0,5.0,3.0,9.0,9.0,0.0,0.0,2.0,0.0,1.0,3.0
2,201501BS70004,1.0,9.0,0.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,30.0,6.0,1686.0,2.0,10.0,1.0,524610.0,181080.0,-0.205590,51.514910,1.0,3.0,1.0,1.0,12/01/2015,2.0,18:08,12.0,E09000020,4.0,415.0,6.0,30.0,2.0,4.0,6.0,0.0,0.0,1.0,4.0,2.0,2.0,0.0,0.0,1.0,1.0,E01002833,1.0,1.0,3.0,2.0,33.0,6.0,3.0,1.0,3.0,0.0,0.0,2.0,0.0,1.0,6.0
3,201501BS70005,1.0,9.0,0.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,524420.0,181080.0,-0.208327,51.514952,1.0,3.0,1.0,1.0,13/01/2015,3.0,07:40,12.0,E09000020,4.0,450.0,6.0,30.0,6.0,4.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,2.0,E01002874,1.0,1.0,3.0,2.0,9.0,2.0,3.0,5.0,1.0,0.0,0.0,2.0,0.0,1.0,2.0
4,201501BS70008,1.0,1.0,0.0,18.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,48.0,8.0,NaN,NaN,NaN,1.0,524630.0,179040.0,-0.206022,51.496572,1.0,2.0,2.0,1.0,09/01/2015,6.0,07:30,12.0,E09000020,3.0,315.0,6.0,30.0,6.0,2.0,3.0,3220.0,0.0,5.0,1.0,2.0,2.0,0.0,0.0,1.0,2.0,E01002814,1.0,1.0,1.0,1.0,48.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0


In [15]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363243 entries, 0 to 363242
Data columns (total 67 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   Accident_Index                               363243 non-null  object 
 1   Vehicle_Reference_df_res                     363243 non-null  float64
 2   Vehicle_Type                                 363181 non-null  float64
 3   Towing_and_Articulation                      362864 non-null  float64
 4   Vehicle_Manoeuvre                            363059 non-null  float64
 5   Vehicle_Location-Restricted_Lane             363067 non-null  float64
 6   Junction_Location                            363159 non-null  float64
 7   Skidding_and_Overturning                     363067 non-null  float64
 8   Hit_Object_in_Carriageway                    363080 non-null  float64
 9   Vehicle_Leaving_Carriageway                  363084 non-nul

In [16]:
numerics = ['int16','int32','int64','float64']

In [17]:
catDF = df_X.select_dtypes(exclude=numerics)

In [18]:
numDF = df_X.select_dtypes(include=numerics)

In [19]:
catDF.head()

,Accident_Index,Sex_of_Driver,Date,Time,Local_Authority_(Highway),LSOA_of_Accident_Location
0,201501BS70001,1.0,12/01/2015,18:45,E09000020,E01002825
1,201501BS70002,1.0,12/01/2015,07:50,E09000020,E01002820
2,201501BS70004,1.0,12/01/2015,18:08,E09000020,E01002833
3,201501BS70005,1.0,13/01/2015,07:40,E09000020,E01002874
4,201501BS70008,1.0,09/01/2015,07:30,E09000020,E01002814


In [20]:
numDF.head()

,Vehicle_Reference_df_res,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,1st_Point_of_Impact,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_Home_Area_Type,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Local_Authority_(District),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,Vehicle_Reference_df,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,1.0,19.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,2143.0,2.0,4.0,NaN,525130.0,180050.0,-0.198465,51.505538,1.0,3.0,1.0,1.0,2.0,12.0,5.0,0.0,6.0,30.0,3.0,4.0,6.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,41.0,7.0,3.0,5.0,1.0,0.0,0.0,2.0,0.0,NaN,NaN
1,1.0,9.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,NaN,NaN,1600.0,1.0,3.0,NaN,526530.0,178560.0,-0.178838,51.491836,1.0,3.0,1.0,1.0,2.0,12.0,6.0,0.0,6.0,30.0,3.0,4.0,3.0,3218.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,24.0,5.0,3.0,9.0,9.0,0.0,0.0,2.0,0.0,1.0,3.0
2,1.0,9.0,0.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,30.0,6.0,1686.0,2.0,10.0,1.0,524610.0,181080.0,-0.205590,51.514910,1.0,3.0,1.0,1.0,2.0,12.0,4.0,415.0,6.0,30.0,2.0,4.0,6.0,0.0,0.0,1.0,4.0,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,33.0,6.0,3.0,1.0,3.0,0.0,0.0,2.0,0.0,1.0,6.0
3,1.0,9.0,0.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,524420.0,181080.0,-0.208327,51.514952,1.0,3.0,1.0,1.0,3.0,12.0,4.0,450.0,6.0,30.0,6.0,4.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,3.0,2.0,9.0,2.0,3.0,5.0,1.0,0.0,0.0,2.0,0.0,1.0,2.0
4,1.0,1.0,0.0,18.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,48.0,8.0,NaN,NaN,NaN,1.0,524630.0,179040.0,-0.206022,51.496572,1.0,2.0,2.0,1.0,6.0,12.0,3.0,315.0,6.0,30.0,6.0,2.0,3.0,3220.0,0.0,5.0,1.0,2.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,1.0,48.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0


In [21]:
# Merge back into a single df
# preparing the X Variables  (Don't forget ot remove the target!!)
X = pd.concat([catDF,numDF],axis=1)
print(X.shape)

(363243, 67)


In [22]:
y = df_X['Sex_of_Driver']

In [23]:
X.drop(['Sex_of_Driver'],axis=1,inplace=True)

In [28]:
# -------------------

In [ ]:
# Display non-numerical features
df_X.select_dtypes(exclude="number").head()